In [ ]:
#create spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext

#create config 

config = SparkConf().setAppName("MySparkApp")
context = SparkContext(conf=config) 

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Datenbanken mit Spark") \
      .getOrCreate() # getOrCreate liefert existierende Session wenn es schon eine gibt


In [ ]:
#test if spark is running 
spark

In [ ]:
## tsv to dataFrame , for csv remove sep t , header option puts first line into header##
df= spark.read.option("sep","\t").csv('bands.tsv',header=True, inferSchema=True)
df.show()

In [ ]:
## merge df and df2
##prepare data##
df2= spark.read.option("sep","\t").csv('bands2.tsv',header=True, inferSchema=True)
# union by name so it doesnt depend on position# 
output= df.unionByName(df2, allowMissingColumns=True).dropDuplicates()
#clean data of nulls
output = output.fillna(0)
output = output.na.fill("")
output = output.drop("name").withColumnRenamed("member","Name")
output.show()

In [ ]:
##write dataframe to file with tab  ##
output.write.option("header",True).option("delimiter","\t").csv()

In [ ]:
output

In [ ]:
## SPARK READ FILES AND USE SOME SQL Basics##
    #read local file
textFile = spark.read.text("new.txt")

# EXAMPLE GET MAX AMOUNT OF WORDS IN maxWords or in ONE LINE output #
maxWords=textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords")))
textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords"))).show(100,False)


## some interesting prompts for SQL
#
#
#x = {
#  size() counts the stuff inside
#  split(input,seperators) splits text at seperator
#  name("ColumName") rename colum
#  max()
#  min()
#  avg()
#  countDistinct()
#}
# .select([x])
# .agg(x) can be used to further use aggregations 
# .distinct(x)
# .collect() puts table into one line 
# .withColumnRenamed(from,to)